<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/examples/Tensorflow_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Basic Tensorflow concepts

Based on examples from this article - https://medium.com/learning-machine-learning/introduction-to-tensorflow-estimators-part-1-39f9eb666bc7


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.12.0


Now let's add some constants

In [ ]:
a = tf.constant(5,name = "a")
b = tf.constant(3, name = "b")
result = tf.add(a,b,name='add_a_b')

And some variables and placeholders

In [ ]:
c = tf.Variable(3,dtype=tf.float32)
d = tf.placeholder(dtype = tf.float32,shape=None)

Next we'll run it and see the output

In [ ]:
sess = tf.Session()
sess.run(result)

8

Now let's initialise the variables and placeholders

In [ ]:
sess.run(tf.global_variables_initializer())
print(sess.run(c,feed_dict = {c:14}))
print(sess.run(d,feed_dict = {d:[1,1,3,5]}))

14.0
[1. 1. 3. 5.]


### Datasets and Iterators

This is a very simple example based on this article - https://kratzert.github.io/2017/06/15/example-of-tensorflows-new-input-pipeline.html

It demonstrates how Datasets, Iterators and Sessions interact. It shows how to fetch data one element at a time from an iterator defined over a Dataset. Since it doesn't include the complications about models and input functions and so on, you can see how the data elements are fetched when the session is run.

In [ ]:
import tensorflow as tf

# Toy data
train_imgs = tf.constant(['train/img1.png', 'train/img2.png',
                          'train/img3.png', 'train/img4.png',
                          'train/img5.png', 'train/img6.png'])
train_labels = tf.constant([0, 0, 0, 1, 1, 1])

val_imgs = tf.constant(['val/img1.png', 'val/img2.png',
                        'val/img3.png', 'val/img4.png'])
val_labels = tf.constant([0, 0, 1, 1])

# create TensorFlow Dataset objects
tr_data = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
val_data = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))

# create TensorFlow Iterator object of type Reinitialisable
iterator = tf.data.Iterator.from_structure(tr_data.output_types,
                                   tr_data.output_shapes)
next_element = iterator.get_next()

# create two initialization ops to switch between the datasets
training_init_op = iterator.make_initializer(tr_data)
validation_init_op = iterator.make_initializer(val_data)

with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(training_init_op)

    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break

    # initialize the iterator on the validation data
    sess.run(validation_init_op)

    # get each element of the validation dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break

(b'train/img1.png', 0)
(b'train/img2.png', 0)
(b'train/img3.png', 0)
(b'train/img4.png', 1)
(b'train/img5.png', 1)
(b'train/img6.png', 1)
End of training dataset.
(b'val/img1.png', 0)
(b'val/img2.png', 0)
(b'val/img3.png', 1)
(b'val/img4.png', 1)
End of training dataset.


### Another Datasets, Iterators, Sessions Example

This example is based on the article - https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html and the related code which is https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/blog_estimators_dataset.py

It shows how to use Datasets and how they work with Iterators and Sessions to fetch one Element of the data at a time.

In [ ]:
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

**Upload the data file**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving iris_training.csv to iris_training.csv


In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "iris_training.csv" with length 2194 bytes


**Training Features**

In [ ]:
# The CSV features in our training & test data
feature_names = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth']

**Input Function**

In [ ]:
def my_input_fn(file_path, perform_shuffle=False, repeat_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])
        label = parsed_line[-1]  # Last element is the label
        del parsed_line[-1]  # Delete last element
        features = parsed_line  # Everything but last elements are the features
        d = dict(zip(feature_names, features)), label
        return d

    dataset = (tf.data.TextLineDataset(file_path)  # Read text file
               .skip(1)  # Skip header row
               .map(decode_csv))  # Transform each elem by applying decode_csv fn
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(4)  # Batch size to use
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

**As an illustration of how Datasets work, Fetch Data one element at at time**

In [ ]:
FILE_TRAIN = "iris_training.csv"
next_batch = my_input_fn(FILE_TRAIN, False)  # Will return Batch_Size elements

In [ ]:
# Now let's try it out, retrieving and printing one batch of data.
# Although this code looks strange, you don't need to understand
# the details.
with tf.Session() as sess:
    first_batch = sess.run(next_batch)
    second_batch = sess.run(next_batch)
print(first_batch)
print(second_batch)

({'SepalLength': array([6.4, 5. , 4.9, 4.9], dtype=float32), 'SepalWidth': array([2.8, 2.3, 2.5, 3.1], dtype=float32), 'PetalLength': array([5.6, 3.3, 4.5, 1.5], dtype=float32), 'PetalWidth': array([2.2, 1. , 1.7, 0.1], dtype=float32)}, array([2, 1, 2, 0], dtype=int32))
({'SepalLength': array([5.7, 4.4, 5.4, 6.9], dtype=float32), 'SepalWidth': array([3.8, 3.2, 3.4, 3.1], dtype=float32), 'PetalLength': array([1.7, 1.3, 1.5, 5.1], dtype=float32), 'PetalWidth': array([0.3, 0.2, 0.4, 2.3], dtype=float32)}, array([0, 0, 0, 2], dtype=int32))


**Train an Estimator using the Input Function**

In [ ]:
# Create the feature_columns, which specifies the input to our model.
# All our input features are numeric, so use numeric_column for each one.
feature_columns = [tf.feature_column.numeric_column(k) for k in feature_names]

# Create a deep neural network regression classifier.
# Use the DNNClassifier pre-made estimator
PATH = "."
classifier = tf.estimator.DNNClassifier(
   feature_columns=feature_columns, # The input features to our model
   hidden_units=[10, 10], # Two layers, each with 10 neurons
   n_classes=3,
   model_dir=PATH) # Path to where checkpoints etc are stored

In [ ]:
# This is where we hook up Datasets with the Estimators! Estimators needs data to perform training, evaluation, and prediction, 
# and it uses the input_fn to fetch the data. Estimators require an input_fn with no arguments, so we create a function with 
# no arguments using lambda, which calls our input_fn with the desired arguments: the file_path, shuffle setting, and repeat_count.

# Train our model, use the previously defined function my_input_fn
# Input to training is a file with training example
# Stop training after 8 iterations of train data (epochs)
classifier.train(
    input_fn=lambda: my_input_fn(FILE_TRAIN, True, 8))

### Linear Regression model

Based on examples from this article - https://medium.com/learning-machine-learning/introduction-to-tensorflow-estimators-part-1-39f9eb666bc7



In [ ]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [ ]:
# Create our model parameters, W and b which stands for weight and bias
W = tf.Variable([3.0],name='weight')
b = tf.Variable([-2.0],name='bias')

In [ ]:
# training data - create two placeholders X and y for the training set and labels. We’ll feed the inputs to them during training
X = tf.placeholder(tf.float32)
# y
Y = tf.placeholder(tf.float32)

In [ ]:
# Define the model for a single feature.
predictions = W*X + b

In [ ]:
# loss function. Here we use sum of squared errors. errors are calculated for each sample or instance, while the loss tells us how good our model is performing on the whole dataset.
loss = tf.reduce_sum(tf.square(predictions-Y))

Optimizers are used for finding the best value for some parameters with respect to some loss function in machine learning models. There’s many kind of optimizers, the one we are using here is called Gradient Descent.We start with a random value for the weight and the bias. The optimizer updates the weight and the bias parameter in a direction(increasing or decreasing the numbers) to minimize the value of the loss. We also provide a learning rate to use as a scale factor while updating.

In [ ]:
# training op
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [ ]:
# create some random values for training set and label and feed them into the model during running the code.
x = [1.1,2.0,3.5,4.8]
y = [2.0,3.4,4.2,5.1]
sess.run(tf.global_variables_initializer())
for train_step in range(2000):
    sess.run(train,{X:x,Y:y})

In [ ]:
# Execute the model to see the final results
weight, bias, loss = sess.run([W, b, loss], {X:x, Y:y})
print("W: %s b: %s loss: %s"%(weight,bias,loss))

W: [0.8422483] b: [1.2378168] loss: 0.2880003
